In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import re

# Load data
df = pd.read_csv("loan_data.csv", low_memory=False)

In [ ]:
# Drop irrelevant columns 
df = df[[
    'loan_amnt', 'term', 'int_rate', 'emp_length', 'home_ownership',
    'annual_inc', 'purpose', 'issue_d', 'loan_status', 'dti',
    'revol_util', 'open_acc', 'pub_rec', 'total_acc'
]]


In [ ]:
#  Target Encoding
df = df[df["loan_status"].isin(["Fully Paid", "Charged Off"])]  # Binary classification
df["target"] = df["loan_status"].map({"Fully Paid": 0, "Charged Off": 1})
df.drop("loan_status", axis=1, inplace=True)